In this assignement, feel free to use the `sparse` module from `scipy`.

Use the cell below for your imports.

In [51]:
import numpy as np
from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix

implement the function `mat_mul_coo` that takes two sparse matrices in `coo` and returns their product.

In [47]:
def mat_mul_coo(A, B):
    if A.shape[1] != B.shape[0]:
        raise ValueError("Incompatible shapes: {} and {}".format(A.shape, B.shape))
    
    # Initialize a dictionary to store the entries of the product matrix
    product_dict = {}
    
    # Compute the product C = AB using the COO format
    for i, j, a_val in zip(A.row, A.col, A.data):
        for k, l, b_val in zip(B.row, B.col, B.data):
            if j == k:
                key = (i, l)
                if key in product_dict:
                    product_dict[key] += a_val * b_val
                else:
                    product_dict[key] = a_val * b_val
    
    # Construct the COO matrix from the computed data
    rows_cols, data = zip(*product_dict.items())
    rows = [rows_cols[0] for rows_cols in rows_cols]
    cols = [rows_cols[1] for rows_cols in rows_cols]
    C = coo_matrix((data, (rows, cols)), shape=(A.shape[0], B.shape[1]))
    
    return C

In [203]:
a = [[1,3,0],[0,0,0],[1,0,13]]
b = [[0,3,0],[2,2,2],[0,0,0]]
A = coo_matrix(a)
B = coo_matrix(b)

In [204]:
print(mat_mul_coo(A, B).toarray())

[[6 9 6]
 [0 0 0]
 [0 3 0]]


check the results.

In [205]:
np.dot(A.toarray(), B.toarray())

array([[6, 9, 6],
       [0, 0, 0],
       [0, 3, 0]])

implement the function `mat_mul_csr` that takes two sparse matrices in `csr` format and returns their product.

In [173]:
def mat_mul_csr(A, B):
    # Compute the product using CSR format
    C = csr_matrix((A.shape[0], B.shape[1]), dtype=A.dtype)
    for i in range(A.shape[0]):
        start = A.indptr[i]
        end = A.indptr[i+1]
        for j in range(start, end):
            k = A.indices[j]
            C[i, :] += A.data[j] * B[k, :]
    return C


In [178]:
a = csr_matrix([[1,3,0],[0,0,0],[1,0,13]])
b = csr_matrix([[0,3,0],[2,2,2],[0,0,0]])

In [179]:
print(mat_mul_csr(a, b).toarray())

[[6 9 6]
 [0 0 0]
 [0 3 0]]


implement a function `solve_lin_sys` that takes a matrix `A` in `csr` format and a vector `b` as a numpy array and solves the system `Ax = b`.

in this function we will use lu decomposition to solve the linear system 

In [197]:
def solve_lin_sys(A_csr, b):
    epsilon = 1e-10
    # Perform LU decomposition of A
    n = A_csr.shape[0]
    L = np.zeros((n, n))
    U = np.zeros((n, n))
    
    # Compute LU decomposition
    for k in range(n):
        U[k, k] = A_csr[k, k] - np.dot(L[k, :k], U[:k, k])
        L[k, k] = 1.0
        for i in range(k+1, n):
            L[i, k] = (A_csr[i, k] - np.dot(L[i, :k], U[:k, k])) / (U[k, k] + epsilon)#to avoic deviding by zero
            U[k, i] = A_csr[k, i] - np.dot(L[k, :k], U[:k, i])

    # Solve L*y = b for y using forward substitution
    y = np.zeros(n)
    for i in range(n):
        y[i] = b[i] - np.dot(L[i, :i], y[:i])
    
    # Solve U*x = y for x using backward substitution
    x = np.zeros(n)
    for i in reversed(range(n)):
        x[i] = (y[i] - np.dot(U[i, i+1:], x[i+1:])) / (U[i, i]+epsilon) #to avoic deviding by zero
    
    # Return solution x
    return x


In [198]:
indptr = np.array([0, 2, 3, 6])
indices = np.array([0, 2, 2, 0, 1, 2])
data = np.array([1, 2, 3, 4, 5, 6])
b = np.array([0,1,1])
A = csr_matrix((data, indices, indptr), shape=(3, 3))

In [199]:
print(solve_lin_sys(A, b))

[-0.66666667  0.33333336  0.33333333]


check the solution 

In [201]:
np.linalg.solve(A.toarray(), b)

array([-0.66666667,  0.33333333,  0.33333333])